In [1]:
import urllib2
import urllib
import sys
import pandas as pd
from bs4 import BeautifulSoup
from cnns.utils import cnn_utils as cu
from cnns.utils import imagenet_utils as imu

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
def get_image_urls(url):
    return urllib2.urlopen(url).read().split('<br>')

In [6]:
evaluation_dir = '/Users/babasarala/Desktop/evaluation'
mat_filepath = '%s/ILSVRC2013_devkit/data/meta_clsloc.mat'%(evaluation_dir)
ground_truth_filepath = '%s/raw_images/ILSVRC2013_clsloc_validation_ground_truth.txt'%(evaluation_dir)
cats_filepath = '/Users/babasarala/technologies/caffe/data/ilsvrc12/synset_words.txt'
collapse_filepath = '/Users/babasarala/repos/cnns/config/\
image_net_categories_animals_screenshots_devices_2016_05_04.json'

val_with_img_idx_url = "http://www.image-net.org/challenges/LSVRC/2012/ori_urls/indexval_w.html"
val_set_csv = '/Users/babasarala/Desktop/ILSVRC2012_validation_set.csv'

val_img_idx_to_cats = imu.load_validation_ground_truth(ground_truth_filepath, mat_filepath)
val_img_idx_to_collapsed_cats = imu.load_validation_ground_truth(ground_truth_filepath, 
                                                                 mat_filepath, 
                                                                 collapse_filepath)
cats_filepath = '/Users/babasarala/technologies/caffe/data/ilsvrc12/synset_words.txt'
wnid_to_imagenet_cats = imu.load_imagenet_cats_ids(cats_filepath)
imagenet_cat_to_wnid = {v:k for k,v in wnid_to_imagenet_cats.iteritems()}

Write ILSVRC2012 validation data to a csv file

In [7]:
data = []
val_url_img_idxs = get_image_urls(val_with_img_idx_url)[:-1]
for val_url_img_idx in val_url_img_idxs:
    tmp = val_url_img_idx.split(' ')
    val_url = ''.join(tmp[:-1]).strip()
    img_idx = int(tmp[-1])
        
    data.append({'wnid': 'n0%i'%(imagenet_cat_to_wnid[val_img_idx_to_cats[img_idx]]),
                'img_url':val_url,
                'imagenet_category': val_img_idx_to_cats[img_idx],
                'internal_category': val_img_idx_to_collapsed_cats[img_idx],
               })
df = pd.DataFrame(data)
df.to_csv(val_set_csv, index=False)

In [8]:
master_url = "http://www.image-net.org/challenges/LSVRC/2012/ori_urls/"
train_set_csv = '/Users/babasarala/Desktop/ILSVRC2012_training_set.csv'

In [18]:
r = urllib.urlopen(master_url).read()
soup = BeautifulSoup(r)

In [10]:
all_links = soup.find_all("a")

In [11]:
train_link_str = []
for link in all_links:
    train_link_str.append(link.get("href"))
train_link_str = train_link_str[2:]

In [12]:
base_url = "http://www.image-net.org/challenges/LSVRC/2012/ori_urls/"
train_links = [base_url+s for s in train_link_str]

In [68]:
wnids = [int(train_link.split('indextrain')[1].split('.html')[0][1:]) for train_link in train_links]

In [14]:
# load some necessary functions
tfm_fcn = cu.get_tfm_fcn_from_json(collapse_filepath)
cat_ids = imu.load_imagenet_cats_ids(cats_filepath)

In [16]:
data = []
for train_link in train_links:
    train_img_urls = get_image_urls(train_link)[:-1] # there's a pesky empty string at the end
    wnid = train_link.split('indextrain')[1].split('.html')[0]
    if wnid[0] == '0':
        wnid = 'n'+train_link.split('indextrain')[1].split('.html')[0]
    else:
        wnid = 'n0'+train_link.split('indextrain')[1].split('.html')[0]
    imagenet_cat = cat_ids[int(wnid[2:])]
    internal_cat = tfm_fcn(imagenet_cat)
    for train_img_url in train_img_urls:
        data.append({'wnid': wnid, 
                     'img_url':train_img_url, 
                     'imagenet_category':imagenet_cat, 
                     'internal_category':internal_cat,
                     })
df = pd.DataFrame(data)

In [17]:
df.to_csv(train_set_csv, index=False)